# CNNbasic Assignment#2

# AlexNet 구현

모델 구현 후 summary로 전체 모델 구조 출력과 주석으로 간단한 설명을 달아주시면 됩니다.

프레임워크는 자유이고, 기본 tensforflow와 pytorch tutorial 사이트를 아래에 첨부해 드립니다.

이 외 각 프레임워크 별 summary 등 추가적인 사용 방법은 구글링으로 찾아주세요!-!

- Tensorflow Tutorial: https://www.tensorflow.org/tutorials?hl=ko

- Pytorch Tutorial: https://tutorials.pytorch.kr/

![image-2.png](attachment:image-2.png)

## 모델 구조
* 5개의 Convolution layer와 3개의 FC layer
* Batchnorm 추가
* Conv1(ReLU) -> Pool1 -> Batchnorm -> Conv2(ReLU) -> Pool2 -> Batchnorm -> Conv3(ReLU) -> Conv4(ReLU) -> Conv5(ReLU) -> Pool3 -> (Flatten) FC1(ReLU) -> FC2(ReLU) -> FC3(->SOFTMAX) 

## Tensorflow(keras)

In [1]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential


n_classes = 1000 # class 개수

model=Sequential()

############## Add Layer ##############

# convolution layer1
model.add(Conv2D(96,(11,11), strides = 4, padding = 'valid', activation = 'relu',input_shape=(227,227,3)))
model.add(MaxPooling2D((3,3), strides = (2,2)))
model.add(BatchNormalization())

# convolution layer2
model.add(Conv2D(256,(5,5),strides = 1, padding = 'same', activation = 'relu', input_shape=(27,27,96)))
model.add(MaxPooling2D((3,3), strides = (2,2)))
model.add(BatchNormalization())

# convolution layer3
model.add(Conv2D(384,(3,3),strides = 1, padding = 'same',  activation = 'relu', input_shape=(13,13,256)))

# convolution layer4
model.add(Conv2D(384,(3,3),strides = 1, padding = 'same', activation = 'relu', input_shape=(13,13,384)))

# convolution layer5
model.add(Conv2D(256,(3,3),strides = 1, padding = 'same',  activation = 'relu', input_shape=(13,13,384)))
model.add(MaxPooling2D((3,3), strides = (2,2)))

#fully-connected layer1
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))

#fully-connected layer2
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))

#fully-connected layer3
model.add(Dense(1000, activation='softmax'))

#######################################

# keras summary
model.summary() # summary code 추가

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 27, 27, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 256)      0         
 2D)                                                             
                                                        

## Pytorch

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


class AlexNet(nn.Module):

    def __init__(self, n_classes = 1000):
        super(AlexNet, self).__init__()
        
        ############## Add Layer ##############
        # convolution layer1
        self.conv1 = nn.Sequential(
          nn.Conv2d(in_channels = 3, out_channels = 96, kernel_size = 11, stride = 4, padding = 0),
          # input size : 3 * 227 * 227
          # input size 정의 : (C, H, W)
          # I'(Size of output image ) = (I-F+2P)/S + 1 = (227-11)/4 + 1 = 55
          # 96 * 55 * 55 feature map 생성
          nn.ReLU(), # 96 * 55 * 55 유지
          nn.MaxPool2d(kernel_size = 3, stride = 2),
          # 55 -> (55-3)/2 + 1 = 27
          # 96x27x27 feature map 생성
          nn.BatchNorm2d(96)) # 96 * 27 * 27 유지
        
        # convolution layer2
        self.conv2 = nn.Sequential(
          nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = 5, stride = 1, padding = 2),
          # input size : 96 * 27 * 27
          # I' = (27-5+2*2)/1 + 1 = 27
          # 256 * 27 * 27 feature map 생성
          nn.ReLU(),  # 256 * 27 * 27 유지
          nn.MaxPool2d(kernel_size = 3, stride = 2),
          # 13 -> (27-3)/2 + 1 = 13
          # 256 * 13 * 13 feature map 생성
          nn.BatchNorm2d(256)) # 256 * 13 * 13 유지
        
        # convolution layer3
        self.conv3 = nn.Sequential(
          nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = 3, stride = 1, padding = 1),
          # 256 * 13 * 13
          # I' = (13-3+2*1)/1 + 1 = 13
          # 384 * 13 * 13 feature map 생성
          nn.ReLU()) # 384 * 13 * 13 유지
        
        # convolution layer4
        self.conv4 = nn.Sequential(
          nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 3, stride = 1, padding = 1),
          # 384 * 13 * 13
          # I' = (13-3+2*1)/1 + 1 = 13
          # 384 * 13 * 13 feature map 생성
          nn.ReLU()) # 384 * 13 * 13 유지
        
        # convolution layer5
        self.conv5 = nn.Sequential(
          nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
          # 384 * 13 * 13
          # I' = (13-3+2*1)/1 + 1 = 13
          # 384 * 13 * 13 feature map 생성
          nn.ReLU(), # 384 * 13 * 13 유지
          nn.MaxPool2d(kernel_size = 3, stride = 2))
          # 6 -> (13-3)/2 + 1 = 6
          # 256 * 6 * 6 feature map 생성
        
        # fully-connected layer1
        self.fc1 = nn.Sequential(
          nn.Flatten(),
          nn.Dropout(0.5),
          nn.Linear(9216, 4096),
          nn.ReLU()
        )

        # fully-connected layer2
        self.fc2 = nn.Sequential(
          nn.Dropout(0.5),
          nn.Linear(4096, 4096),
          nn.ReLU()
        )

        # fully-connected layer3
        self.fc3 = nn.Sequential(
          nn.Linear(4096, n_classes)
        )    
        #######################################

    def forward(self, x):
        
        ############## Add Layer ##############
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = F.log_softmax(out, dim=1) 
        # log_softmax
        # 소프트맥스에 log함수를 취한 것으로 softmax 함수의 Vanishing Gradients(기울기 손실) 문제를 보완함
        #######################################
        
        return out


# pytorch summary
model = AlexNet().to("cuda")
summary(model, (3, 227, 227)) # summary code 추가

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
       BatchNorm2d-4           [-1, 96, 27, 27]             192
            Conv2d-5          [-1, 256, 27, 27]         614,656
              ReLU-6          [-1, 256, 27, 27]               0
         MaxPool2d-7          [-1, 256, 13, 13]               0
       BatchNorm2d-8          [-1, 256, 13, 13]             512
            Conv2d-9          [-1, 384, 13, 13]         885,120
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 384, 13, 13]       1,327,488
             ReLU-12          [-1, 384, 13, 13]               0
           Conv2d-13          [-1, 256, 13, 13]         884,992
             ReLU-14          [-1, 256,